In [115]:
import os
import json
import requests

import pandas as pd

from dotenv import load_dotenv
from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.display import Image

from lets_plot import *
LetsPlot.setup_html()

from sqlalchemy import create_engine
from sqlalchemy import text




## 1. Creating Dataframe 1 - Artists

Now that we have collected our data from the Spotify API, it is imperative that we clean and preprocess the data preparing it for visualisation

In [116]:
# Let's load JSON data from the spotify_artists.json file
with open("../data/spotify_artists.json", "r") as f:
    artist_data = json.load(f)

In [117]:
print(artist_data)

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5'}, 'followers': {'href': None, 'total': 25112468}, 'genres': ['conscious hip hop', 'hip hop', 'north carolina hip hop', 'rap'], 'href': 'https://api.spotify.com/v1/artists/6l3HvQ5sa6mXTsMTB19rO5', 'id': '6l3HvQ5sa6mXTsMTB19rO5', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb4b053c29fd4b317ff825f0dc', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051744b053c29fd4b317ff825f0dc', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1784b053c29fd4b317ff825f0dc', 'height': 160, 'width': 160}], 'name': 'J. Cole', 'popularity': 87, 'type': 'artist', 'uri': 'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'}, 'followers': {'href': None, 'total': 34778236}, 'genres': ['conscious hip hop', 'hip hop', 'rap', 'west coast rap'], 'href': 'https://api.spotify.com

In [118]:
artists = artist_data["artists"]

In [119]:
df_artists = pd.json_normalize(artists)

In [120]:
# We will drop unnecessary columns from the dataframe
df_artists2=df_artists.drop(columns=['images', 'uri', 'external_urls.spotify', 'href','followers.href', 'genres' ])

In [121]:
df_artists2

,id,name,popularity,type,followers.total
0,6l3HvQ5sa6mXTsMTB19rO5,J. Cole,87,artist,25112468
1,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar,96,artist,34778236


In [122]:
    df_artists_neat=df_artists2.rename(columns={
        'type': 'Type',
        'id': 'Spotify_ID',
        'name': 'Name',
        'popularity': 'Popularity',
        'followers.total': 'Total_Followers'
        
})

In [123]:
df_artists_neat

,Spotify_ID,Name,Popularity,Type,Total_Followers
0,6l3HvQ5sa6mXTsMTB19rO5,J. Cole,87,artist,25112468
1,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar,96,artist,34778236


In [124]:
df_artists_neat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Spotify_ID       2 non-null      object
 1   Name             2 non-null      object
 2   Popularity       2 non-null      int64 
 3   Type             2 non-null      object
 4   Total_Followers  2 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 212.0+ bytes


## 2. Creating Dataframe 2 - Spotify Albums

To be able to directly compare the albums of J.Cole and Kendrick Lamar, We have to load the two seperate files, tidy the data and merge the data together to create one, singular dataframe

### 2.1 - J.Cole Album Data

In [125]:
with open("../data/spotify_jcole.json", "r") as f:
     jcole_data = json.load(f)

In [127]:
jcole_albums=jcole_data["albums"]

In [128]:
pd.json_normalize(jcole_albums)

,album_type,total_tracks,is_playable,href,id,images,name,release_date,release_date_precision,type,...,popularity,external_urls.spotify,tracks.href,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total,tracks.items,external_ids.upc
0,album,21,True,https://api.spotify.com/v1/albums/5FP9keIJnlSC...,5FP9keIJnlSCKnkdVOf623,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Born Sinner (Deluxe Version),2013-06-18,day,album,...,80,https://open.spotify.com/album/5FP9keIJnlSCKnk...,https://api.spotify.com/v1/albums/5FP9keIJnlSC...,50,None,0,None,21,[{'artists': [{'external_urls': {'spotify': 'h...,00810045118567
1,album,16,True,https://api.spotify.com/v1/albums/0fhmJYVhW0e4...,0fhmJYVhW0e4i33pCLPA5i,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Cole World: The Sideline Story,2011-09-27,day,album,...,78,https://open.spotify.com/album/0fhmJYVhW0e4i33...,https://api.spotify.com/v1/albums/0fhmJYVhW0e4...,50,None,0,None,16,[{'artists': [{'external_urls': {'spotify': 'h...,00810760035897
2,album,13,True,https://api.spotify.com/v1/albums/0UMMIkurRUmk...,0UMMIkurRUmkruZ3KGBLtG,[{'url': 'https://i.scdn.co/image/ab67616d0000...,2014 Forest Hills Drive,2014-12-09,day,album,...,85,https://open.spotify.com/album/0UMMIkurRUmkruZ...,https://api.spotify.com/v1/albums/0UMMIkurRUmk...,50,None,0,None,13,[{'artists': [{'external_urls': {'spotify': 'h...,00810093551910
3,album,10,True,https://api.spotify.com/v1/albums/3CCnGldVQ90c...,3CCnGldVQ90c26aFATC1PW,[{'url': 'https://i.scdn.co/image/ab67616d0000...,4 Your Eyez Only,2016-12-09,day,album,...,75,https://open.spotify.com/album/3CCnGldVQ90c26a...,https://api.spotify.com/v1/albums/3CCnGldVQ90c...,50,None,0,None,10,[{'artists': [{'external_urls': {'spotify': 'h...,00850498007520
4,album,12,True,https://api.spotify.com/v1/albums/4Wv5UAieM1LD...,4Wv5UAieM1LDEYVq5WmqDd,[{'url': 'https://i.scdn.co/image/ab67616d0000...,KOD,2018-04-20,day,album,...,70,https://open.spotify.com/album/4Wv5UAieM1LDEYV...,https://api.spotify.com/v1/albums/4Wv5UAieM1LD...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00810760031967
5,album,12,True,https://api.spotify.com/v1/albums/4JAvwK4APPAr...,4JAvwK4APPArjIsOdGoJXX,[{'url': 'https://i.scdn.co/image/ab67616d0000...,The Off-Season,2021-05-14,day,album,...,75,https://open.spotify.com/album/4JAvwK4APPArjIs...,https://api.spotify.com/v1/albums/4JAvwK4APPAr...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00810061165149


In [129]:
df_jcole=pd.json_normalize(jcole_albums)

In [130]:
df_jcole

,album_type,total_tracks,is_playable,href,id,images,name,release_date,release_date_precision,type,...,popularity,external_urls.spotify,tracks.href,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total,tracks.items,external_ids.upc
0,album,21,True,https://api.spotify.com/v1/albums/5FP9keIJnlSC...,5FP9keIJnlSCKnkdVOf623,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Born Sinner (Deluxe Version),2013-06-18,day,album,...,80,https://open.spotify.com/album/5FP9keIJnlSCKnk...,https://api.spotify.com/v1/albums/5FP9keIJnlSC...,50,None,0,None,21,[{'artists': [{'external_urls': {'spotify': 'h...,00810045118567
1,album,16,True,https://api.spotify.com/v1/albums/0fhmJYVhW0e4...,0fhmJYVhW0e4i33pCLPA5i,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Cole World: The Sideline Story,2011-09-27,day,album,...,78,https://open.spotify.com/album/0fhmJYVhW0e4i33...,https://api.spotify.com/v1/albums/0fhmJYVhW0e4...,50,None,0,None,16,[{'artists': [{'external_urls': {'spotify': 'h...,00810760035897
2,album,13,True,https://api.spotify.com/v1/albums/0UMMIkurRUmk...,0UMMIkurRUmkruZ3KGBLtG,[{'url': 'https://i.scdn.co/image/ab67616d0000...,2014 Forest Hills Drive,2014-12-09,day,album,...,85,https://open.spotify.com/album/0UMMIkurRUmkruZ...,https://api.spotify.com/v1/albums/0UMMIkurRUmk...,50,None,0,None,13,[{'artists': [{'external_urls': {'spotify': 'h...,00810093551910
3,album,10,True,https://api.spotify.com/v1/albums/3CCnGldVQ90c...,3CCnGldVQ90c26aFATC1PW,[{'url': 'https://i.scdn.co/image/ab67616d0000...,4 Your Eyez Only,2016-12-09,day,album,...,75,https://open.spotify.com/album/3CCnGldVQ90c26a...,https://api.spotify.com/v1/albums/3CCnGldVQ90c...,50,None,0,None,10,[{'artists': [{'external_urls': {'spotify': 'h...,00850498007520
4,album,12,True,https://api.spotify.com/v1/albums/4Wv5UAieM1LD...,4Wv5UAieM1LDEYVq5WmqDd,[{'url': 'https://i.scdn.co/image/ab67616d0000...,KOD,2018-04-20,day,album,...,70,https://open.spotify.com/album/4Wv5UAieM1LDEYV...,https://api.spotify.com/v1/albums/4Wv5UAieM1LD...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00810760031967
5,album,12,True,https://api.spotify.com/v1/albums/4JAvwK4APPAr...,4JAvwK4APPArjIsOdGoJXX,[{'url': 'https://i.scdn.co/image/ab67616d0000...,The Off-Season,2021-05-14,day,album,...,75,https://open.spotify.com/album/4JAvwK4APPArjIs...,https://api.spotify.com/v1/albums/4JAvwK4APPAr...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00810061165149


In [131]:
# Dropping columns
df_jcole2=df_jcole.drop(columns=['is_playable', 'album_type', 'is_playable', 'href', 'images', 'release_date_precision', 'external_urls.spotify', 'tracks.href', 'tracks.limit', 'tracks.limit', 'tracks.offset', 'tracks.previous', 'tracks.items', 'external_ids.upc', 'copyrights', 'genres', 'uri', 'tracks.total', 'tracks.next' ]
)

In [132]:
df_jcole2

,total_tracks,id,name,release_date,type,artists,label,popularity
0,21,5FP9keIJnlSCKnkdVOf623,Born Sinner (Deluxe Version),2013-06-18,album,[{'external_urls': {'spotify': 'https://open.s...,Roc Nation LLC,80
1,16,0fhmJYVhW0e4i33pCLPA5i,Cole World: The Sideline Story,2011-09-27,album,[{'external_urls': {'spotify': 'https://open.s...,Roc Nation LLC,78
2,13,0UMMIkurRUmkruZ3KGBLtG,2014 Forest Hills Drive,2014-12-09,album,[{'external_urls': {'spotify': 'https://open.s...,Roc Nation Records LLC,85
3,10,3CCnGldVQ90c26aFATC1PW,4 Your Eyez Only,2016-12-09,album,[{'external_urls': {'spotify': 'https://open.s...,J. Cole P&D,75
4,12,4Wv5UAieM1LDEYVq5WmqDd,KOD,2018-04-20,album,[{'external_urls': {'spotify': 'https://open.s...,J. Cole P&D,70
5,12,4JAvwK4APPArjIsOdGoJXX,The Off-Season,2021-05-14,album,[{'external_urls': {'spotify': 'https://open.s...,"Dreamville, Inc., Under exclusive license to R...",75


In [133]:
# We shall use the .apply function (not .explode as there is a dictionary) to unnest the artists column and extract data of artist name from the dictionary
df_jcole2['artists'] = df_jcole2['artists'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else None)

The artists column now shows the artist name (J.Cole) for the 6 studio albums

In [134]:
df_jcole2

,total_tracks,id,name,release_date,type,artists,label,popularity
0,21,5FP9keIJnlSCKnkdVOf623,Born Sinner (Deluxe Version),2013-06-18,album,J. Cole,Roc Nation LLC,80
1,16,0fhmJYVhW0e4i33pCLPA5i,Cole World: The Sideline Story,2011-09-27,album,J. Cole,Roc Nation LLC,78
2,13,0UMMIkurRUmkruZ3KGBLtG,2014 Forest Hills Drive,2014-12-09,album,J. Cole,Roc Nation Records LLC,85
3,10,3CCnGldVQ90c26aFATC1PW,4 Your Eyez Only,2016-12-09,album,J. Cole,J. Cole P&D,75
4,12,4Wv5UAieM1LDEYVq5WmqDd,KOD,2018-04-20,album,J. Cole,J. Cole P&D,70
5,12,4JAvwK4APPArjIsOdGoJXX,The Off-Season,2021-05-14,album,J. Cole,"Dreamville, Inc., Under exclusive license to R...",75


In [135]:
    df_jcole_neat=df_jcole2.rename(columns={
        'total_tracks': 'Number_of_Tracks',
        'id': 'Spotify_ID',
        'name': 'Name',
        'release_date': 'Release_Date',
        'type': 'Type',
        'artists': 'Artist',
        'label': 'Label',
        'popularity': 'Popularity'
        
})

In [136]:
df_jcole_neat

,Number_of_Tracks,Spotify_ID,Name,Release_Date,Type,Artist,Label,Popularity
0,21,5FP9keIJnlSCKnkdVOf623,Born Sinner (Deluxe Version),2013-06-18,album,J. Cole,Roc Nation LLC,80
1,16,0fhmJYVhW0e4i33pCLPA5i,Cole World: The Sideline Story,2011-09-27,album,J. Cole,Roc Nation LLC,78
2,13,0UMMIkurRUmkruZ3KGBLtG,2014 Forest Hills Drive,2014-12-09,album,J. Cole,Roc Nation Records LLC,85
3,10,3CCnGldVQ90c26aFATC1PW,4 Your Eyez Only,2016-12-09,album,J. Cole,J. Cole P&D,75
4,12,4Wv5UAieM1LDEYVq5WmqDd,KOD,2018-04-20,album,J. Cole,J. Cole P&D,70
5,12,4JAvwK4APPArjIsOdGoJXX,The Off-Season,2021-05-14,album,J. Cole,"Dreamville, Inc., Under exclusive license to R...",75


When sending the request to extract the data  I accidentaly put the Spotify ID of J.Cole's second album first in the URL so am using .iloc to swap and reorder the first two rows around so the albums are listed accurately by order of release

In [137]:
df_jcole_neat2 = df_jcole_neat.iloc[[1, 0] + list(range(2, len(df_jcole_neat)))]

We need to create a new column listing the albums by release order, this will ultimately form our y-axis

In [138]:
# Use .assign
df_jcole_neat2=df_jcole_neat2.assign(new_column=[1, 2, 3, 4, 5, 6])

In [139]:
df_jcole_neat2 = df_jcole_neat2.rename(columns={
        'new_column': 'Album_by_Release_Order',
        
})

In [140]:
df_jcole_neat2

,Number_of_Tracks,Spotify_ID,Name,Release_Date,Type,Artist,Label,Popularity,Album_by_Release_Order
1,16,0fhmJYVhW0e4i33pCLPA5i,Cole World: The Sideline Story,2011-09-27,album,J. Cole,Roc Nation LLC,78,1
0,21,5FP9keIJnlSCKnkdVOf623,Born Sinner (Deluxe Version),2013-06-18,album,J. Cole,Roc Nation LLC,80,2
2,13,0UMMIkurRUmkruZ3KGBLtG,2014 Forest Hills Drive,2014-12-09,album,J. Cole,Roc Nation Records LLC,85,3
3,10,3CCnGldVQ90c26aFATC1PW,4 Your Eyez Only,2016-12-09,album,J. Cole,J. Cole P&D,75,4
4,12,4Wv5UAieM1LDEYVq5WmqDd,KOD,2018-04-20,album,J. Cole,J. Cole P&D,70,5
5,12,4JAvwK4APPArjIsOdGoJXX,The Off-Season,2021-05-14,album,J. Cole,"Dreamville, Inc., Under exclusive license to R...",75,6


### 2.1 - J.Cole Album Data

Let's now replicate the above code with the Kendrick Lamar album data

In [141]:
with open("../data/spotify_klamar.json", "r") as f:
     klamar_data = json.load(f)

In [143]:
klamar_albums=klamar_data["albums"]

In [144]:
pd.json_normalize(klamar_albums)

,album_type,total_tracks,is_playable,href,id,images,name,release_date,release_date_precision,type,...,popularity,external_urls.spotify,tracks.href,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total,tracks.items,external_ids.upc
0,album,15,True,https://api.spotify.com/v1/albums/1bkN9nIkkCnX...,1bkN9nIkkCnXeG4yitVS1J,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Section.80,2011-07-02,day,album,...,75,https://open.spotify.com/album/1bkN9nIkkCnXeG4...,https://api.spotify.com/v1/albums/1bkN9nIkkCnX...,50,None,0,None,15,[{'artists': [{'external_urls': {'spotify': 'h...,888915968772
1,album,17,True,https://api.spotify.com/v1/albums/748dZDqSZy6a...,748dZDqSZy6aPXKcI9H80u,[{'url': 'https://i.scdn.co/image/ab67616d0000...,"good kid, m.A.A.d city (Deluxe)",2012-10-22,day,album,...,74,https://open.spotify.com/album/748dZDqSZy6aPXK...,https://api.spotify.com/v1/albums/748dZDqSZy6a...,50,None,0,None,17,[{'artists': [{'external_urls': {'spotify': 'h...,00602537439508
2,album,16,True,https://api.spotify.com/v1/albums/7ycBtnsMtyVb...,7ycBtnsMtyVbbwTfJwRjSP,[{'url': 'https://i.scdn.co/image/ab67616d0000...,To Pimp A Butterfly,2015-03-16,day,album,...,81,https://open.spotify.com/album/7ycBtnsMtyVbbwT...,https://api.spotify.com/v1/albums/7ycBtnsMtyVb...,50,None,0,None,16,[{'artists': [{'external_urls': {'spotify': 'h...,00602547289049
3,album,14,True,https://api.spotify.com/v1/albums/4eLPsYPBmXAB...,4eLPsYPBmXABThSJ821sqY,[{'url': 'https://i.scdn.co/image/ab67616d0000...,DAMN.,2017-04-14,day,album,...,89,https://open.spotify.com/album/4eLPsYPBmXABThS...,https://api.spotify.com/v1/albums/4eLPsYPBmXAB...,50,None,0,None,14,[{'artists': [{'external_urls': {'spotify': 'h...,00602557631722
4,album,19,True,https://api.spotify.com/v1/albums/79ONNoS4M9tf...,79ONNoS4M9tfIA1mYLBYVX,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Mr. Morale & The Big Steppers,2022-05-13,day,album,...,84,https://open.spotify.com/album/79ONNoS4M9tfIA1...,https://api.spotify.com/v1/albums/79ONNoS4M9tf...,50,None,0,None,19,[{'artists': [{'external_urls': {'spotify': 'h...,00602448038708
5,album,12,True,https://api.spotify.com/v1/albums/0hvT3yIEysuu...,0hvT3yIEysuuvkK73vgdcW,[{'url': 'https://i.scdn.co/image/ab67616d0000...,GNX,2024-11-22,day,album,...,100,https://open.spotify.com/album/0hvT3yIEysuuvkK...,https://api.spotify.com/v1/albums/0hvT3yIEysuu...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00602475681120


In [145]:
pd.json_normalize(klamar_albums)

,album_type,total_tracks,is_playable,href,id,images,name,release_date,release_date_precision,type,...,popularity,external_urls.spotify,tracks.href,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total,tracks.items,external_ids.upc
0,album,15,True,https://api.spotify.com/v1/albums/1bkN9nIkkCnX...,1bkN9nIkkCnXeG4yitVS1J,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Section.80,2011-07-02,day,album,...,75,https://open.spotify.com/album/1bkN9nIkkCnXeG4...,https://api.spotify.com/v1/albums/1bkN9nIkkCnX...,50,None,0,None,15,[{'artists': [{'external_urls': {'spotify': 'h...,888915968772
1,album,17,True,https://api.spotify.com/v1/albums/748dZDqSZy6a...,748dZDqSZy6aPXKcI9H80u,[{'url': 'https://i.scdn.co/image/ab67616d0000...,"good kid, m.A.A.d city (Deluxe)",2012-10-22,day,album,...,74,https://open.spotify.com/album/748dZDqSZy6aPXK...,https://api.spotify.com/v1/albums/748dZDqSZy6a...,50,None,0,None,17,[{'artists': [{'external_urls': {'spotify': 'h...,00602537439508
2,album,16,True,https://api.spotify.com/v1/albums/7ycBtnsMtyVb...,7ycBtnsMtyVbbwTfJwRjSP,[{'url': 'https://i.scdn.co/image/ab67616d0000...,To Pimp A Butterfly,2015-03-16,day,album,...,81,https://open.spotify.com/album/7ycBtnsMtyVbbwT...,https://api.spotify.com/v1/albums/7ycBtnsMtyVb...,50,None,0,None,16,[{'artists': [{'external_urls': {'spotify': 'h...,00602547289049
3,album,14,True,https://api.spotify.com/v1/albums/4eLPsYPBmXAB...,4eLPsYPBmXABThSJ821sqY,[{'url': 'https://i.scdn.co/image/ab67616d0000...,DAMN.,2017-04-14,day,album,...,89,https://open.spotify.com/album/4eLPsYPBmXABThS...,https://api.spotify.com/v1/albums/4eLPsYPBmXAB...,50,None,0,None,14,[{'artists': [{'external_urls': {'spotify': 'h...,00602557631722
4,album,19,True,https://api.spotify.com/v1/albums/79ONNoS4M9tf...,79ONNoS4M9tfIA1mYLBYVX,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Mr. Morale & The Big Steppers,2022-05-13,day,album,...,84,https://open.spotify.com/album/79ONNoS4M9tfIA1...,https://api.spotify.com/v1/albums/79ONNoS4M9tf...,50,None,0,None,19,[{'artists': [{'external_urls': {'spotify': 'h...,00602448038708
5,album,12,True,https://api.spotify.com/v1/albums/0hvT3yIEysuu...,0hvT3yIEysuuvkK73vgdcW,[{'url': 'https://i.scdn.co/image/ab67616d0000...,GNX,2024-11-22,day,album,...,100,https://open.spotify.com/album/0hvT3yIEysuuvkK...,https://api.spotify.com/v1/albums/0hvT3yIEysuu...,50,None,0,None,12,[{'artists': [{'external_urls': {'spotify': 'h...,00602475681120


In [146]:
df_klamar=pd.json_normalize(klamar_albums)

In [147]:
df_klamar2=df_klamar.drop(columns=['is_playable', 'album_type', 'is_playable', 'href', 'images', 'release_date_precision', 'external_urls.spotify', 'tracks.href', 'tracks.limit', 'tracks.limit', 'tracks.offset', 'tracks.previous', 'tracks.items', 'external_ids.upc', 'copyrights', 'genres', 'uri', 'tracks.total', 'tracks.next' ]
)

In [148]:
df_klamar2

,total_tracks,id,name,release_date,type,artists,label,popularity
0,15,1bkN9nIkkCnXeG4yitVS1J,Section.80,2011-07-02,album,[{'external_urls': {'spotify': 'https://open.s...,Top Dawg Entertainment / Section.80,75
1,17,748dZDqSZy6aPXKcI9H80u,"good kid, m.A.A.d city (Deluxe)",2012-10-22,album,[{'external_urls': {'spotify': 'https://open.s...,Aftermath,74
2,16,7ycBtnsMtyVbbwTfJwRjSP,To Pimp A Butterfly,2015-03-16,album,[{'external_urls': {'spotify': 'https://open.s...,Aftermath,81
3,14,4eLPsYPBmXABThSJ821sqY,DAMN.,2017-04-14,album,[{'external_urls': {'spotify': 'https://open.s...,Aftermath,89
4,19,79ONNoS4M9tfIA1mYLBYVX,Mr. Morale & The Big Steppers,2022-05-13,album,[{'external_urls': {'spotify': 'https://open.s...,pgLang/Top Dawg Entertainment/Aftermath/Inters...,84
5,12,0hvT3yIEysuuvkK73vgdcW,GNX,2024-11-22,album,[{'external_urls': {'spotify': 'https://open.s...,"pgLang, under exclusive license to Interscope ...",100


In [149]:
df_klamar2['artists'] = df_klamar2['artists'].apply(lambda x: x[0]['name'] if isinstance(x, list) and len(x) > 0 else None)

In [150]:
df_klamar2

,total_tracks,id,name,release_date,type,artists,label,popularity
0,15,1bkN9nIkkCnXeG4yitVS1J,Section.80,2011-07-02,album,Kendrick Lamar,Top Dawg Entertainment / Section.80,75
1,17,748dZDqSZy6aPXKcI9H80u,"good kid, m.A.A.d city (Deluxe)",2012-10-22,album,Kendrick Lamar,Aftermath,74
2,16,7ycBtnsMtyVbbwTfJwRjSP,To Pimp A Butterfly,2015-03-16,album,Kendrick Lamar,Aftermath,81
3,14,4eLPsYPBmXABThSJ821sqY,DAMN.,2017-04-14,album,Kendrick Lamar,Aftermath,89
4,19,79ONNoS4M9tfIA1mYLBYVX,Mr. Morale & The Big Steppers,2022-05-13,album,Kendrick Lamar,pgLang/Top Dawg Entertainment/Aftermath/Inters...,84
5,12,0hvT3yIEysuuvkK73vgdcW,GNX,2024-11-22,album,Kendrick Lamar,"pgLang, under exclusive license to Interscope ...",100


In [151]:
    df_klamar_neat=df_klamar2.rename(columns={
        'total_tracks': 'Number_of_Tracks',
        'id': 'Spotify_ID',
        'name': 'Name',
        'release_date': 'Release_Date',
        'type': 'Type',
        'artists': 'Artist',
        'label': 'Label',
        'popularity': 'Popularity'
        
})

In [152]:
 df_klamar_neat

,Number_of_Tracks,Spotify_ID,Name,Release_Date,Type,Artist,Label,Popularity
0,15,1bkN9nIkkCnXeG4yitVS1J,Section.80,2011-07-02,album,Kendrick Lamar,Top Dawg Entertainment / Section.80,75
1,17,748dZDqSZy6aPXKcI9H80u,"good kid, m.A.A.d city (Deluxe)",2012-10-22,album,Kendrick Lamar,Aftermath,74
2,16,7ycBtnsMtyVbbwTfJwRjSP,To Pimp A Butterfly,2015-03-16,album,Kendrick Lamar,Aftermath,81
3,14,4eLPsYPBmXABThSJ821sqY,DAMN.,2017-04-14,album,Kendrick Lamar,Aftermath,89
4,19,79ONNoS4M9tfIA1mYLBYVX,Mr. Morale & The Big Steppers,2022-05-13,album,Kendrick Lamar,pgLang/Top Dawg Entertainment/Aftermath/Inters...,84
5,12,0hvT3yIEysuuvkK73vgdcW,GNX,2024-11-22,album,Kendrick Lamar,"pgLang, under exclusive license to Interscope ...",100


In [153]:
df_klamar_neat=df_klamar_neat.assign(new_column=[1, 2, 3, 4, 5, 6])

In [154]:
df_klamar_neat2 = df_klamar_neat.rename(columns={
        'new_column': 'Album_by_Release_Order',
        
})

In [155]:
df_klamar_neat2

,Number_of_Tracks,Spotify_ID,Name,Release_Date,Type,Artist,Label,Popularity,Album_by_Release_Order
0,15,1bkN9nIkkCnXeG4yitVS1J,Section.80,2011-07-02,album,Kendrick Lamar,Top Dawg Entertainment / Section.80,75,1
1,17,748dZDqSZy6aPXKcI9H80u,"good kid, m.A.A.d city (Deluxe)",2012-10-22,album,Kendrick Lamar,Aftermath,74,2
2,16,7ycBtnsMtyVbbwTfJwRjSP,To Pimp A Butterfly,2015-03-16,album,Kendrick Lamar,Aftermath,81,3
3,14,4eLPsYPBmXABThSJ821sqY,DAMN.,2017-04-14,album,Kendrick Lamar,Aftermath,89,4
4,19,79ONNoS4M9tfIA1mYLBYVX,Mr. Morale & The Big Steppers,2022-05-13,album,Kendrick Lamar,pgLang/Top Dawg Entertainment/Aftermath/Inters...,84,5
5,12,0hvT3yIEysuuvkK73vgdcW,GNX,2024-11-22,album,Kendrick Lamar,"pgLang, under exclusive license to Interscope ...",100,6


### 2.3 - Concatenate to create new df_albums dataframe

In [156]:
df_albums = pd.concat([df_jcole_neat2, df_klamar_neat2], ignore_index=True)

In [157]:
df_albums

,Number_of_Tracks,Spotify_ID,Name,Release_Date,Type,Artist,Label,Popularity,Album_by_Release_Order
0,16,0fhmJYVhW0e4i33pCLPA5i,Cole World: The Sideline Story,2011-09-27,album,J. Cole,Roc Nation LLC,78,1
1,21,5FP9keIJnlSCKnkdVOf623,Born Sinner (Deluxe Version),2013-06-18,album,J. Cole,Roc Nation LLC,80,2
2,13,0UMMIkurRUmkruZ3KGBLtG,2014 Forest Hills Drive,2014-12-09,album,J. Cole,Roc Nation Records LLC,85,3
3,10,3CCnGldVQ90c26aFATC1PW,4 Your Eyez Only,2016-12-09,album,J. Cole,J. Cole P&D,75,4
4,12,4Wv5UAieM1LDEYVq5WmqDd,KOD,2018-04-20,album,J. Cole,J. Cole P&D,70,5
5,12,4JAvwK4APPArjIsOdGoJXX,The Off-Season,2021-05-14,album,J. Cole,"Dreamville, Inc., Under exclusive license to R...",75,6
6,15,1bkN9nIkkCnXeG4yitVS1J,Section.80,2011-07-02,album,Kendrick Lamar,Top Dawg Entertainment / Section.80,75,1
7,17,748dZDqSZy6aPXKcI9H80u,"good kid, m.A.A.d city (Deluxe)",2012-10-22,album,Kendrick Lamar,Aftermath,74,2
8,16,7ycBtnsMtyVbbwTfJwRjSP,To Pimp A Butterfly,2015-03-16,album,Kendrick Lamar,Aftermath,81,3
9,14,4eLPsYPBmXABThSJ821sqY,DAMN.,2017-04-14,album,Kendrick Lamar,Aftermath,89,4


## 4. SQL Database

### 4.1 Artists

SQL Databases are a structured and efficient way to store and query data. We will be using SQLite which is a simple database that comes installed with Python.

It is important we specify the data types of our dataframe columns in Pandas for when we create the database

In [158]:

df_artists_neat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Spotify_ID       2 non-null      object
 1   Name             2 non-null      object
 2   Popularity       2 non-null      int64 
 3   Type             2 non-null      object
 4   Total_Followers  2 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 212.0+ bytes


In [159]:
df_artists_neat['Spotify_ID'].dtypes

dtype('O')

In [160]:
type(df_artists_neat['Spotify_ID'].iloc[0])

str

In [161]:
type(df_artists_neat['Type'].iloc[0])

str

In [162]:
type(df_artists_neat['Name'].iloc[0])

str

In [163]:
# The crucial first step is to create a database engine using SQLAlchemy and connect to the engine
engine = create_engine('sqlite:////files/w10-summative-vedantr98/data/spotify_artist.db', echo=False, isolation_level="AUTOCOMMIT")

with engine.connect() as conn:
    pass

In [164]:

# Let's use SQL's CREATE TABLE statement to manually specify the columns and create our database, let's start with the artists data
create_statement = text("""
CREATE TABLE IF NOT EXISTS artists (
    Name TEXT,
    Spotify_ID TEXT,
    Popularity SMALLINT,
    Type TEXT,
    Total_Followers SMALLINT
);
""")

In [165]:
# Don't forget to insert our data into the existing created table in the database
with engine.connect() as conn:
    conn.execute(text('DROP TABLE IF EXISTS artists;'))
    conn.execute(create_statement)

In [166]:
df_artists_neat.to_sql('artists', engine, if_exists='append', index=False)

2

In [167]:
print(engine)

Engine(sqlite:////files/w10-summative-vedantr98/data/spotify_artist.db)


### 4.2 Albums

Let's do the same for our 2nd set of data on Albums

In [168]:
df_albums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Number_of_Tracks        12 non-null     int64 
 1   Spotify_ID              12 non-null     object
 2   Name                    12 non-null     object
 3   Release_Date            12 non-null     object
 4   Type                    12 non-null     object
 5   Artist                  12 non-null     object
 6   Label                   12 non-null     object
 7   Popularity              12 non-null     int64 
 8   Album_by_Release_Order  12 non-null     int64 
dtypes: int64(3), object(6)
memory usage: 996.0+ bytes


In [169]:
type(df_albums['Release_Date'].iloc[0])

str

In [170]:
create_album_table = text("""
  CREATE TABLE IF NOT EXISTS albums (
    Artist TEXT,
    Name TEXT,
    Spotify_ID TEXT,
    Release_Date TEXT,
    Number_of_Tracks SMALLINT,
    Label TEXT,
    Type TEXT,
    Popularity SMALLINT,
    Album_by_Release_Order SMALLINT
    
);
""")

with engine.connect() as conn:
    conn.execute(create_album_table)

In [171]:
df_albums.to_sql('albums', engine, if_exists='append', index=False)

12

We have now created the database with two tables exhbiting our Artists and Albums dataframes - it should exist under the data folder